<h2>MODEL TESTING</h2>

Script ini digunakan hanya untuk mengetes model-model yang telah dibuat. Tahapan dari mengetes suatu model adalah:
1. Load model yang telah ada. (Saat ini, model yang diloat adalah model v1 yang sudah saya buat).
2. Load x_test dan y_test, untuk menjadi data yang dapat dimasukkan ke model untuk melihat performa model. Bagian ini tidak perlu diubah jika model yang diload berubah.
3. Jalankan block terakhir. Num dapat diubah-ubah untuk melihat trajectory yang berbeda-beda

In [30]:
import tensorflow as tf
import plotly.express as px
import plotly.graph_objects as go
import numpy as np

In [2]:
## Load the model
model = tf.keras.models.load_model('../models/model_v1.keras')

# Show the model architecture
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 normalization (Normalizati  (None, 19, 6)             13        
 on)                                                             
                                                                 
 lstm_3 (LSTM)               (None, 19, 64)            18176     
                                                                 
 lstm_4 (LSTM)               (None, 19, 32)            12416     
                                                                 
 lstm_5 (LSTM)               (None, 16)                3136      
                                                                 
 dense_1 (Dense)             (None, 2)                 34        
                                                                 
Total params: 33775 (131.94 KB)
Trainable params: 33762 (131.88 KB)
Non-trainable params: 13 (56.00 Byte)
______________

In [3]:
## Load numpy test data
x_test = np.load('../x_test.npy')
y_test = np.load('../y_test.npy')

In [4]:
## IMPORTANT: Set up denormalizer
mean = np.array([-5.34506, 91.331474])
variance = np.array([4.855295, 1414.9424])

denormalizer = tf.keras.layers.Normalization(mean=mean, variance=variance, invert=True)

In [64]:
# Num dapat diubah-ubah untuk melihat trajectory yang berbeda-beda
num = 1

# Predict the next coords of entry "num" using the model
num_of_timesteps = len(x_test[0])
pred_coords = denormalizer(model.predict(x_test[num].reshape(1, num_of_timesteps, 6)))
true_coords = denormalizer(y_test[num])

# Print the true and predicted coordinates
print("Prediction:")
print(pred_coords)

print("True Coordinate:")
print(true_coords)


## Map display of True Point vs Predicted Point
past_lat = []
past_lon = []

# Get route coordinates
for point in x_test[num]:
  past_lat.append(point[0])
  past_lon.append(point[1])

# Get prediction coordinates
pred_lat = pred_coords.numpy()[0][0]
pred_lon = pred_coords.numpy()[0][1]

# Get true coordinates from y
true_lat = true_coords.numpy()[0][0]
true_lon = true_coords.numpy()[0][1]


# Build the map
fig = go.Figure()

fig.add_trace(go.Scattermapbox(
  mode="markers+lines",
  lat=past_lat,
  lon=past_lon,
  name="trip",
  marker={'size': 10}))

fig.add_trace(go.Scattermapbox(
  mode="markers",
  lat=[pred_lat],
  lon=[pred_lon],
  name="Predicted Point",
  marker={'size': 15, 'color': 'green'}))

fig.add_trace(go.Scattermapbox(
  mode="markers",
  lat=[true_lat],
  lon=[true_lon],
  name="True Point",
  marker={'size': 15, 'color': 'yellow'}))

fig.update_layout(mapbox_style="open-street-map",)
fig.update_layout(
  margin={"r":0,"t":0,"l":0,"b":0},
  mapbox=dict(
    zoom=10,
    center=go.layout.mapbox.Center(
            lat=-6.2,
            lon=106.816
        ),
  ))
fig.show()

1/1 [==============================] - 0s 26ms/step
Prediction:
tf.Tensor([[ -6.1824007 106.82608  ]], shape=(1, 2), dtype=float32)
True Coordinate:
tf.Tensor([[ -6.183128 106.82115 ]], shape=(1, 2), dtype=float32)
